In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
B,T,C=4,8,2
torch.manual_seed(1337)
xbow=torch.zeros((B,T,C),dtype=float)
x=torch.randint(24,(B,T,C),dtype=float)
#Token can't communicate with future token

for b in range(B):
    for t in range(T):
        xprev=x[b,:t+1]
        print(xprev)
        xbow[b,t]=torch.mean(xprev)


tensor([[ 7., 13.]], dtype=torch.float64)
tensor([[ 7., 13.],
        [20., 16.]], dtype=torch.float64)
tensor([[ 7., 13.],
        [20., 16.],
        [23.,  1.]], dtype=torch.float64)
tensor([[ 7., 13.],
        [20., 16.],
        [23.,  1.],
        [23.,  2.]], dtype=torch.float64)
tensor([[ 7., 13.],
        [20., 16.],
        [23.,  1.],
        [23.,  2.],
        [18.,  2.]], dtype=torch.float64)
tensor([[ 7., 13.],
        [20., 16.],
        [23.,  1.],
        [23.,  2.],
        [18.,  2.],
        [20.,  0.]], dtype=torch.float64)
tensor([[ 7., 13.],
        [20., 16.],
        [23.,  1.],
        [23.,  2.],
        [18.,  2.],
        [20.,  0.],
        [ 9., 22.]], dtype=torch.float64)
tensor([[ 7., 13.],
        [20., 16.],
        [23.,  1.],
        [23.,  2.],
        [18.,  2.],
        [20.,  0.],
        [ 9., 22.],
        [ 6., 10.]], dtype=torch.float64)
tensor([[23., 16.]], dtype=torch.float64)
tensor([[23., 16.],
        [ 9., 19.]], dtype=torch.float64)


In [3]:
t=torch.tril(torch.ones(T,T))
wei=torch.zeros((T,T))
wei=wei.masked_fill(t==0,float('-inf'))
wei=F.softmax(wei,dim=1)

In [4]:
x

tensor([[[ 7., 13.],
         [20., 16.],
         [23.,  1.],
         [23.,  2.],
         [18.,  2.],
         [20.,  0.],
         [ 9., 22.],
         [ 6., 10.]],

        [[23., 16.],
         [ 9., 19.],
         [ 5.,  6.],
         [14., 18.],
         [18.,  9.],
         [ 0.,  3.],
         [17., 20.],
         [23.,  3.]],

        [[ 6., 17.],
         [15., 11.],
         [19.,  1.],
         [ 3., 20.],
         [16.,  0.],
         [21., 15.],
         [18., 11.],
         [ 9., 12.]],

        [[23., 17.],
         [20., 14.],
         [20., 12.],
         [10., 21.],
         [22., 15.],
         [12., 10.],
         [18., 19.],
         [17., 10.]]], dtype=torch.float64)

In [5]:
pe=torch.zeros(65,512)
position=torch.arange(65,dtype=torch.float).unsqueeze(1)
div_term=torch.exp(-torch.arange(0,512,2)*math.log(10000)/512)

In [6]:
a=position*div_term

In [7]:
pe[:,0::2]=torch.sin(position*div_term)
pe

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 8.4147e-01,  0.0000e+00,  8.2186e-01,  ...,  0.0000e+00,
          1.0366e-04,  0.0000e+00],
        [ 9.0930e-01,  0.0000e+00,  9.3641e-01,  ...,  0.0000e+00,
          2.0733e-04,  0.0000e+00],
        ...,
        [-7.3918e-01,  0.0000e+00, -1.1848e-01,  ...,  0.0000e+00,
          6.4271e-03,  0.0000e+00],
        [ 1.6736e-01,  0.0000e+00, -8.8357e-01,  ...,  0.0000e+00,
          6.5307e-03,  0.0000e+00],
        [ 9.2003e-01,  0.0000e+00, -8.8825e-01,  ...,  0.0000e+00,
          6.6344e-03,  0.0000e+00]])

In [8]:
torch.sin(position*div_term)

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 8.4147e-01,  8.2186e-01,  8.0196e-01,  ...,  1.1140e-04,
          1.0746e-04,  1.0366e-04],
        [ 9.0930e-01,  9.3641e-01,  9.5814e-01,  ...,  2.2279e-04,
          2.1492e-04,  2.0733e-04],
        ...,
        [-7.3918e-01, -1.1848e-01,  9.1145e-01,  ...,  6.9066e-03,
          6.6625e-03,  6.4271e-03],
        [ 1.6736e-01, -8.8357e-01,  8.7441e-01,  ...,  7.0180e-03,
          6.7700e-03,  6.5307e-03],
        [ 9.2003e-01, -8.8825e-01,  1.3325e-01,  ...,  7.1294e-03,
          6.8774e-03,  6.6344e-03]])

In [26]:
to_2d=nn.Embedding(5,2)

In [52]:
A=torch.arange(0,4)
B=torch.arange(0,4)

# A=to_2d(A)
# B=to_2d(B)

In [53]:
A,B

(tensor([0, 1, 2, 3]), tensor([0, 1, 2, 3]))

In [57]:
eins=torch.einsum('a,a->a',A,B)
print(eins)

tensor([0, 1, 4, 9])


In [33]:
Product=A@B.T
torch.allclose(eins,Product)


True

In [46]:
theta=1./(10000**(torch.arange(0,512,2)/512)).float()
seq_idx=torch.arange(0,300).float()
idx_theta = torch.einsum('n,d->nd', seq_idx, theta)
idx_theta2 = torch.cat([idx_theta, idx_theta], dim=1)


In [50]:
theta.shape,seq_idx.shape,idx_theta.shape

(torch.Size([256]), torch.Size([300]), torch.Size([300, 256]))

In [56]:
seq_idx

tensor([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
         12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,
         24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,
         36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,
         48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,
         60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,
         72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,
         84.,  85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,
         96.,  97.,  98.,  99., 100., 101., 102., 103., 104., 105., 106., 107.,
        108., 109., 110., 111., 112., 113., 114., 115., 116., 117., 118., 119.,
        120., 121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
        132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
        144., 145., 146., 147., 148., 14